In [1]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import copy
import random
import time
import logging
import math
import sys
import os
import numpy as np
import pandas as pd
#import pygame
#from pygame import gfxdraw
from ipdb import set_trace as debug

In [2]:
TIME_SLOT = 120 # sec
WINDOW = 5
TIME_DECISION = 1 # sec

NUM_SENSOR = 4
NUM_EDGE = 4
POS_SENSOR = np.array([[100,0],[0,200],[300,0],[0,-400]])
POS_EDGE = np.array([[-250,250],[250,250],[250,-250],[-250,-250]])
AREA_EDGE = np.array([[-500,500],[-500,500]])

DATA_SIZE_MB = 1 # MB

TX_POWER_DBM = 25
NOISE_POWER_DBM = -80
BANDWIDTH = 5 * (10 ** 6)
USER_DENSITY = 4*10**(-6) # node/m^2 (interfering user density)

DATA_RATE_CONST = 2 * (10 ** 6) # 2Mbps ==> TX time = 0.5
ERR_CONST = 0.01

HUM_CORR = 0.0005483


DATA_SIZE = DATA_SIZE_MB * (8 * 10**6)
TX_POWER = 10 ** (TX_POWER_DBM/10)
NOISE_POWER = 10 ** (NOISE_POWER_DBM/10)

EPISODE_LENGTH = 30

#### Initialization

In [3]:
TX_TIME = np.array([[26.374, 61.964,  7.367, 22.896]])
print(TX_TIME)

AoI = np.zeros(NUM_SENSOR)
AoI_peak = np.zeros(NUM_SENSOR)
est_err = np.zeros(NUM_SENSOR)
print(AoI)

[[26.374 61.964  7.367 22.896]]
[0. 0. 0. 0.]


In [4]:
# STP: array
STP = np.array([[1,1,0,0]])

# action: array
action = np.array([[1,0,1,0]])

# real_veh: array
real_veh_step = np.array([[1,1,1,1]])

# sensing
sensing = np.multiply(action, real_veh_step)
print(sensing)

[[1 0 1 0]]


In [5]:
AoI_prev = copy.copy(AoI)
print(AoI_prev)

[0. 0. 0. 0.]


In [6]:
# TIME STEP 0
#sensing_0 = [1,0,1,0], STP=[1,1,0,0] ==> [1,0,0,0]

print(AoI_prev)
for i in range(NUM_SENSOR):
    if sensing[0,i] == 1:
        if STP[0,i] == 1:
            AoI[i] = TIME_SLOT - TIME_DECISION
            AoI_tmp = AoI_prev[i] + TIME_DECISION + TX_TIME[0,i]
            AoI_peak[i] = max(AoI[i], AoI_tmp)
        else:
            AoI[i] += TIME_SLOT
            AoI_peak[i] = AoI[i]
    else:
        AoI[i] += TIME_SLOT
        AoI_peak[i] = AoI[i]


print("Initial AoI:{}".format(AoI_prev))
print("AoI:{}".format(AoI))
print("Peak AoI:{}".format(AoI_peak))

[0. 0. 0. 0.]
Initial AoI:[0. 0. 0. 0.]
AoI:[119. 120. 120. 120.]
Peak AoI:[119. 120. 120. 120.]


In [7]:
# TIME STEP 1
#sensing_0 = [1,0,1,0], STP=[1,1,0,0] ==> [1,0,0,0]
#sensing_1 = [1,1,0,0], STP=[1,1,0,0] ==> [1,1,0,0]

AoI_prev = copy.copy(AoI)
print(AoI_prev)

STP = np.array([[1,1,0,0]])
sensing = np.array([[1,1,0,0]])

print(AoI_prev)
for i in range(NUM_SENSOR):
    if sensing[0,i] == 1:
        if STP[0,i] == 1:
            AoI[i] = TIME_SLOT - TIME_DECISION
            AoI_tmp = AoI_prev[i] + TIME_DECISION + TX_TIME[0,i]
            AoI_peak[i] = max(AoI[i], AoI_tmp)
        else:
            AoI[i] += TIME_SLOT
            AoI_peak[i] = AoI[i]
    else:
        AoI[i] += TIME_SLOT
        AoI_peak[i] = AoI[i]

print("Initial AoI:{}".format(AoI_prev))
print("AoI:{}".format(AoI))
print("Peak AoI:{}".format(AoI_peak))

[119. 120. 120. 120.]
[119. 120. 120. 120.]
Initial AoI:[119. 120. 120. 120.]
AoI:[119. 119. 240. 240.]
Peak AoI:[146.374 182.964 240.    240.   ]


In [8]:
# TIME STEP 2
#sensing_0 = [1,0,1,0], STP=[1,1,0,0] ==> [1,0,0,0]
#sensing_1 = [1,1,0,0], STP=[1,1,0,0] ==> [1,1,0,0]
#sensing_2 = [0,1,1,0], STP=[1,1,1,0] ==> [0,1,1,0]

AoI_prev = copy.copy(AoI)
print(AoI_prev)

STP = np.array([[1,1,1,0]])
sensing = np.array([[0,1,1,0]])

print(AoI_prev)
for i in range(NUM_SENSOR):
    if sensing[0,i] == 1:
        if STP[0,i] == 1:
            AoI[i] = TIME_SLOT - TIME_DECISION
            AoI_tmp = AoI_prev[i] + TIME_DECISION + TX_TIME[0,i]
            AoI_peak[i] = max(AoI[i], AoI_tmp)
        else:
            AoI[i] += TIME_SLOT
            AoI_peak[i] = AoI[i]
    else:
        AoI[i] += TIME_SLOT
        AoI_peak[i] = AoI[i]

print(sensing)
print(STP)


print("Initial AoI:{}".format(AoI_prev))
print("AoI:{}".format(AoI))
print("Peak AoI:{}".format(AoI_peak))

[119. 119. 240. 240.]
[119. 119. 240. 240.]
[[0 1 1 0]]
[[1 1 1 0]]
Initial AoI:[119. 119. 240. 240.]
AoI:[239. 119. 119. 360.]
Peak AoI:[239.    181.964 248.367 360.   ]


In [9]:
# TIME STEP 3
#sensing_0 = [1,0,1,0], STP=[1,1,0,0] ==> [1,0,0,0]
#sensing_1 = [1,1,0,0], STP=[1,1,0,0] ==> [1,1,0,0]
#sensing_2 = [0,1,1,0], STP=[1,1,1,0] ==> [0,1,1,0]
#sensing_2 = [0,1,0,1], STP=[1,1,0,1] ==> [0,1,0,1]

AoI_prev = copy.copy(AoI)
print(AoI_prev)

STP = np.array([[1,1,0,1]])
sensing = np.array([[0,1,0,1]])

print(AoI_prev)
for i in range(NUM_SENSOR):
    if sensing[0,i] == 1:
        if STP[0,i] == 1:
            AoI[i] = TIME_SLOT - TIME_DECISION
            AoI_tmp = AoI_prev[i] + TIME_DECISION + TX_TIME[0,i]
            AoI_peak[i] = max(AoI[i], AoI_tmp)
        else:
            AoI[i] += TIME_SLOT
            AoI_peak[i] = AoI[i]
    else:
        AoI[i] += TIME_SLOT
        AoI_peak[i] = AoI[i]

print("Initial AoI:{}".format(AoI_prev))
print("AoI:{}".format(AoI))
print("Peak AoI:{}".format(AoI_peak))

[239. 119. 119. 360.]
[239. 119. 119. 360.]
Initial AoI:[239. 119. 119. 360.]
AoI:[359. 119. 239. 119.]
Peak AoI:[359.    181.964 239.    383.896]


In [12]:
print("Initial AoI:{}".format(AoI_prev))
print("AoI:{}".format(AoI))
print("Peak AoI:{}".format(AoI_peak))

Initial AoI:[239. 119. 119. 360.]
AoI:[359. 119. 239. 119.]
Peak AoI:[359.    181.964 239.    383.896]


In [16]:
AoI_TH = 170
eta = np.zeros(NUM_SENSOR)

for i in range(NUM_SENSOR):
    if AoI_prev[i] < AoI_TH:
        if AoI_peak[i] > AoI_TH:
            eta[i] = AoI_peak[i] - AoI_TH
        else:
            eta[i] = 0
    else:
        eta[i] = AoI_peak[i] - AoI_prev[i]

print("eta:{}".format(eta))

eta:[120.     11.964  69.     23.896]


In [36]:
ERR_CONST = 0.01
HUM_CORR = 0.005483

AoI_TH = np.exp(1-ERR_CONST) / (2*HUM_CORR)

print("AoI Threshold:{}".format(AoI_TH))

AoI Threshold:245.41623858738484


In [43]:
np.sum(TIME_SLOT-eta) / NUM_SENSOR - 60 * np.sum(STP)

-116.215

In [40]:
np.sum(STP)

3

In [47]:
rand_act_lst = np.eye(NUM_SENSOR)
print(rand_act_lst)
act_idx = np.random.randint(3, size = NUM_SENSOR)
print(act_idx)
init_act = rand_act_lst[act_idx] 
print(init_act)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[1 0 1 1]
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]
